In [ ]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
class Image_set:
    def __init__(self):
        self.num_data = 31461
        self.squared_img_path = ".\\flickr30k_squared"
        self.blured_img_path = ".\\flickr30k_blured"
        self.indicator = 0
        self.index = [i for i in range(31461)]
        self.index = np.random.permutation(self.index)
        
    def next_batch(self,batch_size):
            
        end_indicator = self.indicator + batch_size
        if end_indicator >= self.num_data:
            self.index = np.random.permutation(self.index)
            self.indicator = 0
            end_indicator = self.indicator + batch_size
        assert end_indicator < self.num_data
        
        
        batch_img_index = self.index[self.indicator: end_indicator]
        batch_squared_img = []
        batch_blured_img = []
        
        for img_index in batch_img_index:
            squared_img_path = os.path.join(self.squared_img_path,"%d.jpg" % img_index)
            blured_img_path = os.path.join(self.blured_img_path,"%d.jpg" % img_index)
            
            squared_img = np.array(cv2.imread(squared_img_path,1),dtype = np.float32)
            blured_img = np.array(cv2.imread(blured_img_path,1),dtype = np.float32)
            squared_img = (squared_img-127.5)/127.5
            blured_img = (blured_img-127.5)/127.5
            
            batch_squared_img.append(squared_img)
            batch_blured_img.append(blured_img)
        
        self.indicator = end_indicator
        
        if batch_size ==1:#用于测试
            batch_squared_img = []
            batch_blured_img = []
            squared_img_path = os.path.join(self.squared_img_path,"%d.jpg" % 0)
            blured_img_path = os.path.join(self.blured_img_path,"%d.jpg" % 0)
            squared_img = np.array(cv2.imread(squared_img_path,1),dtype = np.float32)
            blured_img = np.array(cv2.imread(blured_img_path,1),dtype = np.float32)
            squared_img = (squared_img-127.5)/127.5
            blured_img = (blured_img-127.5)/127.5
            batch_squared_img.append(squared_img)
            batch_blured_img.append(blured_img)
        return batch_squared_img,batch_blured_img



In [ ]:
x_image = tf.placeholder(tf.float32,[None,256,256,3])
y_image = tf.placeholder(tf.float32,[None,256,256,3])    

conv1 = tf.layers.conv2d(x_image,64,(3,3),padding='same',activation=None,name='conv1')#256*256*3 -> 256*256*64
bn1 = tf.nn.relu(tf.layers.batch_normalization(conv1))
pooling1 = tf.layers.max_pooling2d(bn1,(2,2),(2,2),name = 'pool1') #256*256*64 -> 128*128*64

conv2 = tf.layers.conv2d(pooling1,128,(3,3),padding='same',activation=None,name='conv2')#128*128*64 -> 128*128*128
bn2 = tf.nn.relu(tf.layers.batch_normalization(conv2))
pooling2 = tf.layers.max_pooling2d(bn2,(2,2),(2,2),name = 'pool2')#128*128*128 ->64*64*128

conv3 = tf.layers.conv2d(pooling2,256,(3,3),padding='same',activation=None,name='econv3')#64*64*128 -> 64*64*256
bn3 = tf.nn.relu(tf.layers.batch_normalization(conv3))
pooling3 = tf.layers.max_pooling2d(bn3,(2,2),(2,2),name = 'pool3')#64*64*256 -> 32*32*256

conv4 = tf.layers.conv2d(pooling3,512,(3,3),padding='same',activation=None,name='conv4')#32*32*256 -> 32*32*512
bn4 = tf.nn.relu(tf.layers.batch_normalization(conv4))
pooling4 = tf.layers.max_pooling2d(bn4,(2,2),(2,2),name = 'pool4')#32*32*512 -> 16*16*512

conv5 = tf.layers.conv2d(pooling4,512,(3,3),padding='same',activation=None,name='conv5')#16*16*512 -> 16*16*512
bn5 = tf.nn.relu(tf.layers.batch_normalization(conv5))
pooling5 = tf.layers.max_pooling2d(bn5,(2,2),(2,2),name = 'pool5')#16*16*512 -> 8*8*512

conv6 = tf.layers.conv2d(pooling5,512,(3,3),padding='same',activation=None,name='conv6')#8*8*512 -> 8*8*512
bn6 = tf.nn.relu(tf.layers.batch_normalization(conv6))
pooling6 = tf.layers.max_pooling2d(bn6,(2,2),(2,2),name = 'pool6')#8*8*512 -> 4*4*512

conv7 = tf.layers.conv2d(pooling6,512,(3,3),padding='same',activation=None,name='conv7')#4*4*512 -> 4*4*512
bn7 = tf.nn.relu(tf.layers.batch_normalization(conv7))
pooling7 = tf.layers.max_pooling2d(bn7,(2,2),(2,2),name = 'pool7')#4*4*512 -> 2*2*512

conv8 = tf.layers.conv2d(pooling7,512,(3,3),padding='same',activation=None,name='conv8')#2*2*512 -> 2*2*512
bn8 = tf.nn.relu(tf.layers.batch_normalization(conv8))
pooling8 = tf.layers.max_pooling2d(bn8,(2,2),(2,2),name = 'pool8')#2*2*512 -> 1*1*512

conv1_transpose = tf.layers.conv2d_transpose(pooling8, 512,[3,3],strides=(2,2),padding='SAME')#1*1*512 -> 2*2*512
bn1_transpose = tf.nn.relu(tf.layers.batch_normalization(conv1_transpose))
bn1_transpose_res = bn1_transpose + 0.5*pooling7

conv2_transpose = tf.layers.conv2d_transpose(bn1_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#2*2*512 -> 4*4*512
bn2_transpose = tf.nn.relu(tf.layers.batch_normalization(conv2_transpose))
bn2_transpose_res = bn2_transpose + 0.25*pooling6

conv3_transpose = tf.layers.conv2d_transpose(bn2_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#4*4*512 -> 8*8*512
bn3_transpose = tf.nn.relu(tf.layers.batch_normalization(conv3_transpose))
bn3_transpose_res = bn3_transpose + 0.12*pooling5

conv4_transpose = tf.layers.conv2d_transpose(bn3_transpose_res, 512,[3,3],strides=(2,2),padding='SAME')#8*8*512 -> 16*16*512
bn4_transpose = tf.nn.relu(tf.layers.batch_normalization(conv4_transpose))
bn4_transpose_res = bn4_transpose + 0.06*pooling4

conv5_transpose = tf.layers.conv2d_transpose(bn4_transpose_res, 256,[3,3],strides=(2,2),padding='SAME')#16*16*512->32*32*256
bn5_transpose = tf.nn.relu(tf.layers.batch_normalization(conv5_transpose))
bn5_transpose_res = bn5_transpose + 0.03*pooling3

conv6_transpose = tf.layers.conv2d_transpose(bn5_transpose_res, 128,[3,3],strides=(2,2),padding='SAME')#32*32*256->64*64*128
bn6_transpose = tf.nn.relu(tf.layers.batch_normalization(conv6_transpose))
bn6_transpose_res = bn6_transpose + 0.015*pooling2

conv7_transpose = tf.layers.conv2d_transpose(bn6_transpose_res, 64,[3,3],strides=(2,2),padding='SAME')#64*64*128->128*128*64
bn7_transpose = tf.nn.relu(tf.layers.batch_normalization(conv7_transpose))
bn7_transpose_res = bn7_transpose #+ 0.0000*pooling1

y_image_predict = tf.nn.relu(tf.layers.conv2d_transpose(bn7_transpose_res, 3,[3,3],strides=(2,2),padding='SAME'))#128*128*64->256*256*3

loss = tf.reduce_mean(tf.square(y_image-y_image_predict))
tf.summary.scalar('loss',loss)
global_step = tf.Variable(0,trainable=False)#训练的次数

In [ ]:
batch_size = 32
learning_rate = 0.001
train_steps = 100000
img_set = Image_set()


model_dir = '.\\run_model'
log_dir ='.\\run_log'
test_dir = '.\\test_img'
saver = tf.train.Saver()


with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=global_step)
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir,sess.graph)
    sess.run(init_op)
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(model_dir, ckpt_name))
        
    test  = True    
    #for step in range(train_steps):
    for step in range(1):
        if not test:
            start_time = time.time()
            batch_squared_img,batch_blured_img = img_set.next_batch(batch_size)
            global_step_val,losses,losses_merged,_= sess.run([global_step,loss,merged,train_op],
                                                          feed_dict={x_image:batch_blured_img , y_image:batch_squared_img})
        
            use_time = time.time()-start_time
            print("global_step=",global_step_val," losses=",losses," time=%02fs"%use_time)
        
            if global_step_val%50 ==0:
                writer.add_summary(losses_merged,global_step_val)
                saver.save(sess,os.path.join(model_dir,'ckp-%06d' %global_step_val))
                print('model saved to ckp-%06d' %global_step_val)
                test = True
            
        else:    #每50步测试一下图片生成情况
            batch_squared_img,batch_blured_img = img_set.next_batch(1)
            global_step_val,generated_imgs = sess.run([global_step,y_image_predict],
                                                        feed_dict={x_image:batch_blured_img , y_image:batch_squared_img})
            #不训练执行train_op的时候，global_step_val不会+1
            img = generated_imgs[-1]
            img = (img+1)*127.5
            img = np.array(img,dtype = np.uint8)
            out_put_path = os.path.join(test_dir,"%06d.jpg" %global_step_val)
            cv2.imwrite(out_put_path,img)
            #cv2.imshow('a',img)
            #cv2.waitKey()
            test = False
            
            
        

            
            